In [ ]:
# Author: Sanjeewa Senanayaka & Pooja Seth
# Phase 2
# Client program that allows a member to register or login and play the game

from socket import *
import os
import sys
from Crypto import Random
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
import getpass
import warnings
import re
import struct
from time import time

# Function to create a connection to the IAM server
def createConnection():
    serverName = "localhost"
    serverPort = 62843
    clientSocket = socket(AF_INET,SOCK_STREAM)
    clientSocket.connect((serverName, serverPort))
    return clientSocket

# Function to create a connection to the Cloud server
def createConnectionToCloud():
    serverName = "35.236.106.211"
    serverPort = 3426
    clientSocket = socket(AF_INET,SOCK_STREAM)
    clientSocket.connect((serverName, serverPort))
    return clientSocket

# Function that print the menu and waits for a valid input
def menu():
    option = 0
    print("Welcome to hangman! \nPlease choose the option you desire below. \n1. Login \n2. Register \n3. Exit")

    while(option != '1' and option != '2' and option != '3'):
        option = input();
        if(option != '1' and option != '2' and option != '3'):
           warnings.warn("Invalid input! Please try again")
        
    return option

# Function to pad the message to be sent to the server to a multiplicative of 16
def pad(message):
    length = len(message)
    if(length %16 != 0):
        message += ' '* (16-length%16)
        
    return message


# The game functionality function
def playGame(cloudSocket,username,paid_status):
    if(paid_status == "1"):
        level = input("Enter 1 for basic game or 2 for advanced game: ")
        levelBytes = bytes(level,'utf-8')
        cloudSocket.send(levelBytes);
    else:
        level = "1"
        levelBytes = bytes(level,'utf-8')
        cloudSocket.send(levelBytes);
        
    code = cloudSocket.recv(1024)
    code = code.decode('utf-8')
    hint = code.split("\t")[0].strip()
    print("Hint: ", hint)
    chosenword = code.split("\t")[1].strip()
    blank_word = code.split("\t")[2].strip()
    print("Current guessed word is:" , blank_word)
    attempts = 6
    while attempts > 0:
        print(("\nYou have {} attempts remaining.").format(attempts))
        guess = str(input("\nPlease  select a letter between A-Z: ")).lower()
        guessBytes = bytes(guess,'utf-8')
        
        #send guess to server
        cloudSocket.send(guessBytes)
        updatedList = cloudSocket.recv(1024)
        list1 = str(updatedList,'utf-8')
        print("Current guessed word is:", list1)
        
        if guess not in chosenword:
            attempts -= 1
            print("Your guess is not in the word")
            continue
            
        if list1 == chosenword:
            print("Your word was: ", chosenword)
            print("You win")
#             win_message = "game"+"\t"+username+"\t"+str(10)
#             win  = bytes(win_message,'utf-8')
#             cloudSocket.send(win)
            attempts = -1
            print("Do you want to play again? ")
            response = input("> ").lower()
            responseBytes = bytes(response,'utf-8')
            cloudSocket.send(responseBytes)
            if(response == "yes"):
                playGame(cloudSocket,username,paid_status)
            else:
                complete = complete = "Thank you for playing Hangman"
                print(complete)
                break
            
    if attempts == 0:
        print("Sorry you lost the game")
        print("Your word is: ",chosenword)
        print("Do you want to play again? Enter Yes or No")
        response = input("> ").lower()
        responseBytes = bytes(response,'utf-8')
        cloudSocket.send(responseBytes)
        if(response == "yes"):
            playGame(cloudSocket,username,paid_status)
        else:
            complete = "Thank you for playing Hangman"
            print(complete)

            

            
# Takes in login info and confirms it with the IAM server
def loginRequest(clientSocket,session_key,IV):
    
    AES_object = AES.new(session_key, AES.MODE_CBC, IV)
    status = "Fail"

    userName = ""
    password = ""

    username_length = 0
    password_length = 0
    
    #checking if a username or password is entered
    while(username_length == 0 or password_length ==0):
        userName = input("User name:")
        password = getpass.getpass("Password: ")

        username_length = len(userName)
        password_length = len(password)
        if(password_length == 0):
            warnings.warn("Please enter a password")
        elif(username_length ==0):
            warnings.warn("Please enter the user name")


    #login message to be sent to the IAM server
    login_message = "Login" + "\t" + userName + "\t" + password + "\n" 
    padded_message = pad(login_message)
    ciphered_message = AES_object.encrypt(padded_message)
    clientSocket.send(ciphered_message)

    #Checking with the IAM server if the login was a success
    encyrpyted_login_request = clientSocket.recv(1024)
    AES_object_dec = AES.new(session_key, AES.MODE_CBC, IV)
    decrypted_login_request = AES_object_dec.decrypt(encyrpyted_login_request).decode("ascii")
    status = decrypted_login_request.split("\t")[0].strip()
    
    print("Status: "+ decrypted_login_request)

    #if it was a successful login do the following
    if(status == "Success"):
        print("Sending to cloud...")
        cookie = decrypted_login_request.split("\t")[1].strip()
        
        #Create a connection to the cloud and send a message requesting the public Key
        cloudSocket = createConnectionToCloud()
        pub = "publickey"
        padded_pub = pad(pub).encode()
        cloudSocket.send(padded_pub)
        public_key = cloudSocket.recv(1024)
        print("public key received")
        
        #Create a session key using time + random number generator = always unique
        curTime = int(time())
        b = struct.pack(">i", curTime)
        session_key_cloud = get_random_bytes(12)+b
        print("session key is : ", session_key_cloud)

        #RSA encrypt the session key using the Public Key received
        keyPub = RSA.importKey(public_key)
        cipher = PKCS1_OAEP.new(keyPub)
        ciphered_session = cipher.encrypt(session_key_cloud)
        print("sending session key...")
        cloudSocket.send(ciphered_session)

        #RSA encrypt the IV using the Public Key received
        IV_cloud = get_random_bytes(16)
        print("IV is: ", IV_cloud)
        cipher = PKCS1_OAEP.new(keyPub)
        ciphered_IV = cipher.encrypt(IV_cloud)
        print("sending IV...")
        cloudSocket.send(ciphered_IV)

        confirmation = cloudSocket.recv(1024).decode()
    
        #confirmation if session key was accepted
        if(confirmation != "Success"):
            print("Session Key is too old! Possible replay attack! Connection force closing...")
            
        else:
            print("status: ", decrypted_login_request)
            paid_status = decrypted_login_request.split("\t")[2].strip()
            
            #prepare message with username, cookie, and paid status to send to the cloud
            successful_login = "cookie"+ "\t" + userName+"\t"+cookie+"\t"+paid_status
            padded_success_login= pad(successful_login)
            AES_object = AES.new(session_key_cloud, AES.MODE_CBC, IV_cloud)
            ciphered_padded_success_login = AES_object.encrypt(padded_success_login)
            cloudSocket.send(ciphered_padded_success_login)
        
            #Need to access the game if it worked else it needs to retry
            approved = cloudSocket.recv(1024)
            approved = approved.decode()
            print("APPROVED IS: ", approved)
            if(approved == "Success"):
                print("Game starting...")       
                playGame(cloudSocket,userName,paid_status)
                cloudSocket.close()
                
            else:
                print("An error occured. Please try again")
                cloudSocket.close()
        
    else:
        loginRequest(clientSocket,session_key,IV)
    

#Takes in user information to register the account. This information is input to the IAM db and the username is sent 
#to the cloud to be stored
def registerRequest(clientSocket,session_key,IV):
    successful_username = "Fail"
    
    first_name = input("First Name:")
    second_name = input("Second Name:")
    
    #check if the email is in a valid email format (email@yahoo.com)
    email_correctness = False
    while(email_correctness == False):
        email = input("Email Address:")
        if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
            warnings.warn("Invalid email address")
        else:
            email_correctness = True
            
    #check with the IAM server if the username is currently taken
    while(successful_username != "Success"):
        username = input("Username: ")
        test_username = "Username"+ "\t" + username
        padded_user = pad(test_username)
        AES_object = AES.new(session_key, AES.MODE_CBC, IV)
        ciphered_message = AES_object.encrypt(padded_user)
        clientSocket.send(ciphered_message)
        encyrpyted_user_request = clientSocket.recv(1024)
        AES_object_dec = AES.new(session_key, AES.MODE_CBC, IV)
        decrypted_user_request = AES_object_dec.decrypt(encyrpyted_user_request).decode("ascii")
        print("dec: "+ decrypted_user_request)
        successful_username = decrypted_user_request.split("\t")[0].strip()
    
    #Checking if the password selected by the user matches a certain set of criterion
    password_check_flag = 0
    password_check = "Fail"
    
    while password_check != "Success":
        password = input("Password: ")
        while True: 
            if (len(password)<10): 
                password_check_flag = -1
                break
            elif not re.search("[a-z]", password): 
                password_check_flag = -1
                break
            elif not re.search("[A-Z]", password): 
                password_check_flag = -1
                break
            elif not re.search("[0-9]", password): 
                password_check_flag = -1
                break
            elif re.search("\s", password): 
                password_check_flag = -1
                break
            elif not re.search("[_@$]", password): 
                password_check_flag = -1
                break
            else:
                password_check = "Success"
                password_check_flag = 0
                break

        if password_check_flag == -1: 
            print("Password should match the following criteria!")
            print("Minimum 8 characters.")
            print("The alphabets must be between [a-z]")
            print("At least one alphabet should be of Upper Case [A-Z]")
            print("At least 1 number or digit between [0-9].")
            print("At least 1 character from [ _ or @ or $ ].") 
    
    #Ask the user if they want to pay for the game
    paid = input("Pay for game? (0- NO, 1-YES): ")
    
    #Prepare message to send to IAM server
    AES_object = AES.new(session_key, AES.MODE_CBC, IV)
    register_message = "Registration"+"\t"+username+"\t"+password+"\t"+first_name+"\t"+second_name+"\t"+email+"\t"+paid
    padded_register = pad(register_message)
    ciphered_message = AES_object.encrypt(padded_register)
    print("Sending registration message to the IAM server...")
    clientSocket.send(ciphered_message)

    #Receive if the registration was successful
    encyrpyted_login_response = clientSocket.recv(1024)
    AES_object_dec = AES.new(session_key, AES.MODE_CBC, IV)
    decrypted_login_response = AES_object_dec.decrypt(encyrpyted_user_request).decode("ascii")

    if(decrypted_login_response.strip() == "Success"):
        print("Succesful registration!!")
    else:
        print("Error occured during registration")

    
def main():
    
    option = menu()
    
    #Create connection to the IAM Server and receive the public key
    clientSocket = createConnection()
    public_key = clientSocket.recv(1024)
    
    #Create a session key consisting of the current time and random bytes
    curTime = int(time())
    b = struct.pack(">i", curTime)
    session_key = get_random_bytes(12)+b
    
    #RSA encrypt the session key using the public key and send it to IAM
    keyPub = RSA.importKey(public_key)
    cipher = PKCS1_OAEP.new(keyPub)
    ciphered_session = cipher.encrypt(session_key)
    clientSocket.send(ciphered_session)
    
    #RSA encrypt the IV using the public key and send it to IAM
    IV = get_random_bytes(16)
    cipher = PKCS1_OAEP.new(keyPub)
    ciphered_IV = cipher.encrypt(IV)
    clientSocket.send(ciphered_IV)
    
    #Wait for confirmation if the session key was accepted
    confirmation = clientSocket.recv(1024).decode()
    
    if(confirmation != "Success"):
        print("Session Key is too old! Possible replay attack! Connection force closing...")
        clientSocket.close()
    
    else:
        if(option == '1'):
            loginRequest(clientSocket,session_key,IV)

        elif(option == '2'):
            registerRequest(clientSocket,session_key,IV)

        elif(option == '3'):
            print("Bye bye!!")
            exit()
    
main()